In [1]:
# fundemental modules
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import time
import pprint
from tqdm.notebook import trange

from ddqn import DDQNAgent # type: ignore
from utils import record_videos, load_config, show_videos
from reward import _reward
from obs import observation_shape

best_score = -100000.0
score_history = []
episode_lens = []
avg_history = []
std_history = []
avg_history_100 = []



2023-11-27 22:24:24.081316: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-27 22:24:24.105258: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 22:24:24.105282: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 22:24:24.105295: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 22:24:24.109965: I tensorflow/core/platform/cpu_feature_g

In [7]:
data = load_config()
env = gym.make('racetrack-v0', render_mode = 'rgb_array')
env.configure(data) # type: ignore
# pprint.pprint(env.config) # type: ignore
(obs, info), done = env.reset(), False

#observation config
proc = observation_shape(obs,info,2)
proc.reset()
input = proc.get_input()
print(input.shape)


(26,)


/home/o/.local/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(


In [8]:
agent = DDQNAgent(alpha=0.001, gamma=0.9, epsilon=1.0, obs_shape=input.shape,
                  batch_size=256, epsilon_dec=0.9993, epsilon_end=0.05, mem_size=100000,
                  min_mem_size=600, replace_target=1000, learning_rate=0.0003)
print("Agent is initialized.")


Agent is initialized.


2023-11-27 22:28:52.953387: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-27 22:28:53.071645: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-27 22:28:53.071918: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [11]:
# main loop
for episode in trange(100, desc='Test episodes'):

        # proc.reset()
        (observation, info), done = env.reset(), False
        proc.update_input(observation, info)
        observation = proc.get_input()
        # observation = observation.flatten()

        episode_reward = 0
        episode_len = 0

        while not done:
            
            action, action_index = agent.get_action(observation, deterministic=False)
            new_observation, reward, done, truncated, new_info = env.step(action=[action])
            # reward = _reward(new_info,new_observation)
            # new_observation = new_observation.flatten()
            
            proc.update_input(new_observation, info)
            new_observation = proc.get_input()

            episode_reward += reward # type: ignore
            episode_len +=1
            
            if new_info["rewards"]["on_road_reward"] == False:
                 done = True

            agent.remember(state=observation, action=action_index, done=done,
                            reward=reward, new_state=new_observation)
            agent.train()

            observation = new_observation

        episode_lens.append(episode_len)

        score_history.append(episode_reward)
        avg_score = np.mean(score_history)
        avg_history.append(avg_score)
        std_score = np.std(score_history)
        std_history.append(std_score)

        avg_score_100 = np.mean(score_history[-100:])
        avg_history_100.append(avg_score_100)

        # if avg_score_100 > best_score:
            # best_score = avg_score
        if episode_len > 50:
            agent.save_model(0000000)

        agent.tensorboard.update_stats(episode_rew = episode_reward,
                                       average_rew =avg_score,
                                       average_100_reward = avg_score_100,
                                       std_rew=std_score,
                                       epsilon=agent.epsilon,
                                       episode_len = episode_len)

        print('episode ', episode, 'score %.1f' % episode_reward, 'ep len', episode_len,
              'avg score %.1f' % avg_score, 'avg_score_100 %.1f' %avg_score_100,'std score %.1f' % std_score)

Test episodes:   0%|          | 0/100 [00:00<?, ?it/s]

episode  0 score 17.9 ep len 33 avg score 30.3 avg_score_100 64.3 std score 30.6
-----saving models------
episode  1 score 108.4 ep len 151 avg score 30.6 avg_score_100 65.0 std score 30.9
episode  2 score 12.7 ep len 18 avg score 30.5 avg_score_100 64.8 std score 30.9
-----saving models------
episode  3 score 105.9 ep len 145 avg score 30.8 avg_score_100 65.3 std score 31.1
Target Updated
Target Updated
Target Updated
Target Updated
Target Updated
Target Updated
Target Updated
Target Updated
Target Updated
Target Updated
Target Updated
Target Updated
Target Updated
Target Updated
Target Updated
Target Updated
-----saving models------
episode  4 score 122.0 ep len 160 avg score 31.1 avg_score_100 66.0 std score 31.5
-----saving models------
episode  5 score 139.7 ep len 190 avg score 31.4 avg_score_100 66.8 std score 32.1
-----saving models------
episode  6 score 102.9 ep len 132 avg score 31.6 avg_score_100 67.3 std score 32.3
episode  7 score 15.5 ep len 23 avg score 31.6 avg_score_1

In [14]:
n_steps = sum(episode_lens)
print(n_steps)
agent.save_model(1)


27026
-----saving models------


In [20]:
# env = record_videos(env)
from gymnasium.wrappers import RecordVideo

data = load_config()
env = gym.make('racetrack-v0', render_mode = 'rgb_array')
env.configure(data) # type: ignore

env = RecordVideo(env, video_folder="racetrack_ppo/videos", episode_trigger=lambda e: True)
env.unwrapped.set_record_video_wrapper(env)

agent.load_model()
# main loop
for episode in trange(2, desc='Test episodes'):

        (observation, info), done = env.reset(), False
        proc.update_input(observation, info)
        observation = proc.get_input()

        while not done:
            
            action, action_index = agent.get_action(observation, deterministic=True)
            new_observation, reward, done, truncated, new_info = env.step(action=[action])
            # new_observation = new_observation.flatten()
            
            proc.update_input(new_observation, info)
            new_observation = proc.get_input()
            
            if new_info["rewards"]["on_road_reward"] == False:
                 done = True

            # agent.remember(state=observation, action=action_index, done=done,
            #                 reward=reward, new_state=new_observation)
            # agent.train()

            observation = new_observation

        

-----loading models------


/home/o/.local/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(


Test episodes:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
env.close()

Moviepy - Building video /home/o/Documents/hihgway/ddqn/racetrack_ppo/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /home/o/Documents/hihgway/ddqn/racetrack_ppo/videos/rl-video-episode-0.mp4



TypeError: must be real number, not NoneType